In [14]:
from typing import Optional
from torch import nn
from torch.nn import functional as F
import torch

class SpaceTranslator(nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        hidden_layers,
        activation,
        dropout_rate,
        scaling=None,
        centering=None
    ):
        super().__init__()

        self.scaling = scaling
        self.centering = centering

        layers = []
        last = input_dim

        for hidden in hidden_layers:
            layers += [
                nn.Linear(last, hidden),
                # nn.LayerNorm(hidden),
                nn.BatchNorm1d(hidden),
                activation(),
                nn.Dropout(dropout_rate)
            ]
            last = hidden

        layers.append(nn.Linear(last, output_dim))
        self.net = nn.Sequential(*layers)

        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1/0.07))

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_uniform_(module.weight, a=0, mode='fan_in', nonlinearity='relu')
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
        elif isinstance(module, nn.LayerNorm) or isinstance(module, nn.BatchNorm1d):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(self, x):
      return self.net(x)



In [16]:
from pathlib import Path
import numpy as np
import pandas as pd

'''Code from https://github.com/Mamiglia/challenge'''

def mrr(pred_indices: np.ndarray, gt_indices: np.ndarray) -> float:
    """
    Compute Mean Reciprocal Rank (MRR)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries (top-K)
        gt_indices: (N,) array of ground truth indices
    Returns:
        mrr: Mean Reciprocal Rank
    """
    reciprocal_ranks = []
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i] == gt_indices[i])[0]
        if matches.size > 0:
            reciprocal_ranks.append(1.0 / (matches[0] + 1))
        else:
            reciprocal_ranks.append(0.0)
    return np.mean(reciprocal_ranks)


def recall_at_k(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int) -> float:
    """Compute Recall@k
    Args:
        pred_indices: (N, N) array of top indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        recall: Recall@k
    """
    recall = 0
    for i in range(len(gt_indices)):
        if gt_indices[i] in pred_indices[i, :k]:
            recall += 1
    recall /= len(gt_indices)
    return recall

import numpy as np

def ndcg(pred_indices: np.ndarray, gt_indices: np.ndarray, k: int = 100) -> float:
    """
    Compute Normalized Discounted Cumulative Gain (NDCG@k)
    Args:
        pred_indices: (N, K) array of predicted indices for N queries
        gt_indices: (N,) array of ground truth indices
        k: number of top predictions to consider
    Returns:
        ndcg: NDCG@k
    """
    ndcg_total = 0.0
    for i in range(len(gt_indices)):
        matches = np.where(pred_indices[i, :k] == gt_indices[i])[0]
        if matches.size > 0:
            rank = matches[0] + 1
            ndcg_total += 1.0 / np.log2(rank + 1)  # DCG (IDCG = 1)
    return ndcg_total / len(gt_indices)



@torch.inference_mode()
def evaluate_retrieval(translated_embd, image_embd, gt_indices, max_indices = 99, batch_size=100):
    """Evaluate retrieval performance using cosine similarity
    Args:
        translated_embd: (N_captions, D) translated caption embeddings
        image_embd: (N_images, D) image embeddings
        gt_indices: (N_captions,) ground truth image indices for each caption
        max_indices: number of top predictions to consider
    Returns:
        results: dict of evaluation metrics

    """
    # Compute similarity matrix
    if isinstance(translated_embd, np.ndarray):
        translated_embd = torch.from_numpy(translated_embd).float()
    if isinstance(image_embd, np.ndarray):
        image_embd = torch.from_numpy(image_embd).float()

    n_queries = translated_embd.shape[0]
    device = translated_embd.device

    # Prepare containers for the fragments to be reassembled
    all_sorted_indices = []
    l2_distances = []

    # Process in batches - the narrow gate approach
    for start_idx in range(0, n_queries, batch_size):
        batch_slice = slice(start_idx, min(start_idx + batch_size, n_queries))
        batch_translated = translated_embd[batch_slice]
        batch_img_embd = image_embd[batch_slice]

        # Compute similarity only for this batch
        batch_similarity = batch_translated @ batch_img_embd.T

        # Get top-k predictions for this batch
        batch_indices = batch_similarity.topk(k=max_indices, dim=1, sorted=True).indices.numpy()
        all_sorted_indices.append(gt_indices[batch_slice][batch_indices])

        # Compute L2 distance for this batch
        batch_gt = gt_indices[batch_slice]
        batch_gt_embeddings = image_embd[batch_gt]
        batch_l2 = (batch_translated - batch_gt_embeddings).norm(dim=1)
        l2_distances.append(batch_l2)

    # Reassemble the fragments
    sorted_indices = np.concatenate(all_sorted_indices, axis=0)

    # Apply the sacred metrics to the whole
    metrics = {
        'mrr': mrr,
        'ndcg': ndcg,
        'recall_at_1': lambda preds, gt: recall_at_k(preds, gt, 1),
        'recall_at_3': lambda preds, gt: recall_at_k(preds, gt, 3),
        'recall_at_5': lambda preds, gt: recall_at_k(preds, gt, 5),
        'recall_at_10': lambda preds, gt: recall_at_k(preds, gt, 10),
        'recall_at_50': lambda preds, gt: recall_at_k(preds, gt, 50),
    }

    results = {
        name: func(sorted_indices, gt_indices)
        for name, func in metrics.items()
    }

    l2_dist = torch.cat(l2_distances, dim=0).mean().item()
    results['l2_dist'] = l2_dist

    return results

def eval_on_val(x_val: np.ndarray, y_val: np.ndarray, model: nn.Module, device) -> dict:
    gt_indices = torch.arange(len(y_val))

    model.eval()

    with torch.inference_mode():
        translated = model(x_val.to(device)).to('cpu')

    results = evaluate_retrieval(translated, y_val, gt_indices)

    return results


def generate_submission(model: nn.Module, test_path: Path, output_file="submission-dirmodel.csv", device=None):
    test_data = np.load(test_path)
    sample_ids = test_data['captions/ids']
    test_embds = test_data['captions/embeddings']
    test_embds = torch.from_numpy(test_embds).float()

    with torch.no_grad():
        pred_embds = model(test_embds.to(device)).cpu()

    print("Generating submission file...")

    if isinstance(pred_embds, torch.Tensor):
        pred_embds = pred_embds.cpu().numpy()

    df_submission = pd.DataFrame({'id': sample_ids, 'embedding': pred_embds.tolist()})

    df_submission.to_csv(output_file, index=False, float_format='%.17g')
    print(f"✓ Saved submission to {output_file}")

    return df_submission

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

def info_nce_loss(dir_preds, img_targets, logit_scale: float):
    dir_preds = F.normalize(dir_preds, dim=-1)
    img_targets = F.normalize(img_targets, dim=-1)

    logit_scale = torch.clamp(logit_scale, max=np.log(100))

    logits = (dir_preds @ img_targets.T) * logit_scale.exp()
    labels = torch.arange(logits.size(0), device=logits.device)

    loss_t2i = F.cross_entropy(logits, labels)
    loss_i2t = F.cross_entropy(logits.T, labels)

    return 0.5 * (loss_t2i + loss_i2t)

def info_nce_loss_hard(dir_preds, img_targets, logit_scale, k=256):
    # Normalize (important)
    dir_preds = F.normalize(dir_preds, dim=-1)
    img_targets = F.normalize(img_targets, dim=-1)

    logit_scale = torch.clamp(logit_scale, max=np.log(100))
    logits = torch.matmul(dir_preds, img_targets.t()) * logit_scale.exp()  # (B, B)
    B = logits.size(0)
    device = logits.device

    labels = torch.arange(B, device=device)

    # Mask out positives 
    diag_mask = torch.eye(B, device=device).bool()
    neg_logits = logits.masked_fill(diag_mask, float('-inf'))  


    # Get top-k hardest negatives per row 
    hard_vals_t2i, hard_idx_t2i = torch.topk(neg_logits, k, dim=1)  # (B, k)
    # Get positive logits
    pos_logits = logits[torch.arange(B, device=device), labels].unsqueeze(1)  # (B, 1)

    logits_t2i_hard = torch.cat([pos_logits, hard_vals_t2i], dim=1)  # (B, 1+k)
    labels_t2i = torch.zeros(B, dtype=torch.long, device=device)  # positive is index 0

    loss_t2i = F.cross_entropy(logits_t2i_hard, labels_t2i)

    neg_logits_T = logits.t().masked_fill(diag_mask, float('-inf'))  # (B, B)
    hard_vals_i2t, hard_idx_i2t = torch.topk(neg_logits_T, k, dim=1)
    pos_logits_i2t = logits.t()[torch.arange(B, device=device), labels].unsqueeze(1)
    logits_i2t_hard = torch.cat([pos_logits_i2t, hard_vals_i2t], dim=1)
    labels_i2t = torch.zeros(B, dtype=torch.long, device=device)
    loss_i2t = F.cross_entropy(logits_i2t_hard, labels_i2t)

    return 0.5 * (loss_t2i + loss_i2t)


def l2_regularization(model, lambda_l2):
    l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
    return lambda_l2 * l2_norm


def train_model_direction(
    model: SpaceTranslator,
    model_path: Path,
    train_dataset: TensorDataset,
    val_dataset: TensorDataset,
    batch_size: int,
    epochs: int,
    lr: float,
    patience: int,
    reg_lambda: float = 0.03
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print(f"🚀 Using device: {device}")

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=3,
        threshold=0.005,
        min_lr=1e-6
    )

    best_mrr = float('-inf')
    no_improvements = 0

    for epoch in range(1, epochs + 1):
        model.train()
        running_train_loss = 0.0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for X_batch, y_batch in progress_bar:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # X_batch = F.normalize(X_batch, p=2, dim=-1)
            # y_batch = F.normalize(y_batch, p=2, dim=-1)

            optimizer.zero_grad()
            outputs = model(X_batch)

            loss = info_nce_loss(outputs, y_batch, model.logit_scale)
            #loss += l2_regularization(model, reg_lambda)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        train_loss = running_train_loss / len(train_loader)

        # Validation phase
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                # X_batch = F.normalize(X_batch, p=2, dim=-1)
                # y_batch = F.normalize(y_batch, p=2, dim=-1)

                outputs = model(X_batch)

                loss = info_nce_loss(outputs, y_batch, model.logit_scale)
                #loss += l2_regularization(model, reg_lambda)

                running_val_loss += loss.item()

        val_loss = running_val_loss / len(val_loader)

        print(f"📘 Epoch {epoch:03d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

        # Optional: external validation/test
        results = test(val_dataset, model, device)
        print(results, 'Logit scale:', model.logit_scale.exp().item())
        mrr = results['mrr']

        scheduler.step(mrr)

        # Early stopping
        if mrr > best_mrr:
            best_mrr = mrr
            no_improvements = 0

            Path(model_path).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), model_path)
            print(f"💾 Saved new best model (mrr={mrr:.6f})")
        else:
            no_improvements += 1
            if no_improvements >= patience:
                print("⏹ Early stopping triggered.")
                break

    print(f"✅ Training complete. Best mrr: {mrr:.6f}")

    return model


def get_data(data_path: Path):
    data = np.load(data_path)
    caption_embeddings = data['captions/embeddings']
    image_embeddings = data['images/embeddings']
    caption_labels = data['captions/label']
    data.close()

    X_abs, y_abs = torch.tensor(caption_embeddings), torch.tensor(image_embeddings[np.argmax(caption_labels, axis=1)])

    return X_abs, y_abs

def get_datasets(X_abs, y_abs) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    print('Texts shape', X_abs.shape)
    print('Images shape', y_abs.shape)

    dataset = TensorDataset(X_abs, y_abs)
    train_dataset, val_dataset = random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

    return train_dataset, val_dataset

def test(val_dataset: TensorDataset, model: nn.Module, device):
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
    for x_val, y_val in val_loader:
        results = eval_on_val(x_val, y_val, model=model, device=device)
    return results

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = 1024
output_dim = 1536
hidden_layers=[1256, 1656]
dropout_rate = 0.5

batch_size= 2048
lr=0.01
epochs= 250
patience = 8

data_path= '/content/drive/MyDrive/dav/train.npz'
test_path= '/content/drive/MyDrive/dav/test.clean.npz'

save_path = './models/dir-model.pth'

In [8]:
x, y = get_data(data_path)

train_dataset, val_dataset = get_datasets(x, y)

Texts shape torch.Size([125000, 1024])
Images shape torch.Size([125000, 1536])


In [36]:
model_args = {
    'input_dim': x.size(1),
    'output_dim': y.size(1),
    'hidden_layers': hidden_layers,
    'dropout_rate': dropout_rate,
    'activation': nn.ReLU
}

model = SpaceTranslator(**model_args).to(device)

train_model_direction(model, save_path, train_dataset, val_dataset, batch_size, epochs, lr, patience)

print('Finished training. Now testing using best model...')

state = torch.load(save_path)
model.load_state_dict(state)
results = test(val_dataset, model, device)
print("Test Results:", results)

🚀 Using device: cuda


📘 Epoch 001 | Train Loss: 4.891643 | Val Loss: 3.718127 | LR: 1.00e-02
{'mrr': np.float64(0.7448052281678955), 'ndcg': np.float64(0.8060800068067063), 'recall_at_1': 0.6124, 'recall_at_3': 0.85448, 'recall_at_5': 0.91964, 'recall_at_10': 0.97016, 'recall_at_50': 0.99888, 'l2_dist': 277.0684814453125} Logit scale: 23.2886962890625
💾 Saved new best model (mrr=0.744805)


📘 Epoch 002 | Train Loss: 3.565430 | Val Loss: 2.843952 | LR: 1.00e-02
{'mrr': np.float64(0.828902700497973), 'ndcg': np.float64(0.8705432968094536), 'recall_at_1': 0.7294, 'recall_at_3': 0.91752, 'recall_at_5': 0.95708, 'recall_at_10': 0.98412, 'recall_at_50': 0.99908, 'l2_dist': 260.9161682128906} Logit scale: 36.78339767456055
💾 Saved new best model (mrr=0.828903)


📘 Epoch 003 | Train Loss: 2.977001 | Val Loss: 2.432778 | LR: 1.00e-02
{'mrr': np.float64(0.8639403120457971), 'ndcg': np.float64(0.8972404897036886), 'recall_at_1': 0.78128, 'recall_at_3': 0.93876, 'recall_at_5': 0.969, 'recall_at_10': 0.98888, 'recall_at_50': 0.99932, 'l2_dist': 280.1271057128906} Logit scale: 50.50896453857422
💾 Saved new best model (mrr=0.863940)


📘 Epoch 004 | Train Loss: 2.642725 | Val Loss: 2.211477 | LR: 1.00e-02
{'mrr': np.float64(0.8805852670009757), 'ndcg': np.float64(0.9098967208232419), 'recall_at_1': 0.80644, 'recall_at_3': 0.94872, 'recall_at_5': 0.9734, 'recall_at_10': 0.99088, 'recall_at_50': 0.99952, 'l2_dist': 288.9298095703125} Logit scale: 62.784698486328125
💾 Saved new best model (mrr=0.880585)


...

📘 Epoch 048 | Train Loss: 0.941972 | Val Loss: 1.542526 | LR: 7.81e-05
{'mrr': np.float64(0.9297132672773586), 'ndcg': np.float64(0.9469215497453004), 'recall_at_1': 0.88492, 'recall_at_3': 0.971, 'recall_at_5': 0.98408, 'recall_at_10': 0.993, 'recall_at_50': 0.99936, 'l2_dist': 188.7519989013672} Logit scale: 100.76211547851562


📘 Epoch 049 | Train Loss: 0.937992 | Val Loss: 1.542267 | LR: 7.81e-05
{'mrr': np.float64(0.9297524359036611), 'ndcg': np.float64(0.9469500398656241), 'recall_at_1': 0.88504, 'recall_at_3': 0.97112, 'recall_at_5': 0.98408, 'recall_at_10': 0.99288, 'recall_at_50': 0.9994, 'l2_dist': 188.34521484375} Logit scale: 100.76211547851562


📘 Epoch 050 | Train Loss: 0.939952 | Val Loss: 1.542368 | LR: 3.91e-05
{'mrr': np.float64(0.929758754678339), 'ndcg': np.float64(0.9469551543782836), 'recall_at_1': 0.88504, 'recall_at_3': 0.97104, 'recall_at_5': 0.98408, 'recall_at_10': 0.99296, 'recall_at_50': 0.9994, 'l2_dist': 188.19631958007812} Logit scale: 100.76211547851562


📘 Epoch 051 | Train Loss: 0.940660 | Val Loss: 1.542644 | LR: 3.91e-05
{'mrr': np.float64(0.9297640232634562), 'ndcg': np.float64(0.9469627405692421), 'recall_at_1': 0.88496, 'recall_at_3': 0.97132, 'recall_at_5': 0.98412, 'recall_at_10': 0.99296, 'recall_at_50': 0.9994, 'l2_dist': 188.56350708007812} Logit scale: 100.76211547851562


📘 Epoch 052 | Train Loss: 0.936226 | Val Loss: 1.542225 | LR: 3.91e-05
{'mrr': np.float64(0.9298726110844082), 'ndcg': np.float64(0.9470423322592552), 'recall_at_1': 0.88512, 'recall_at_3': 0.97108, 'recall_at_5': 0.98404, 'recall_at_10': 0.99296, 'recall_at_50': 0.9994, 'l2_dist': 188.78956604003906} Logit scale: 100.76211547851562
⏹ Early stopping triggered.
✅ Training complete. Best mrr: 0.929873
Finished training. Now testing using best model...
Test Results: {'mrr': np.float64(0.9299668128234295), 'ndcg': np.float64(0.9471116568627964), 'recall_at_1': 0.8854, 'recall_at_3': 0.97124, 'recall_at_5': 0.98416, 'recall_at_10': 0.993, 'recall_at_50': 0.9994, 'l2_dist': 189.603515625}


In [37]:
generate_submission(model, Path(test_path), output_file="dav.csv", device=device)

Generating submission file...
✓ Saved submission to dav.csv


,id,embedding
0,1,"[0.7686144709587097, 2.381528854370117, 1.0353..."
1,2,"[-4.1902289390563965, 0.8161473870277405, -0.6..."
2,3,"[-1.2787296772003174, -1.561556100845337, -2.2..."
3,4,"[11.185599327087402, -4.036659240722656, -3.34..."
4,5,"[6.494488716125488, 8.447858810424805, 1.51297..."
...,...,...
1495,1496,"[-0.43608617782592773, -5.116342067718506, 5.4..."
1496,1497,"[-0.33950138092041016, 2.236297130584717, 9.56..."
1497,1498,"[2.6319985389709473, -4.231359004974365, -1.67..."
1498,1499,"[-3.7497448921203613, -3.935755729675293, -2.3..."


In [38]:
torch.save(model, "full_model.pth")